In [5]:
import cProfile
from statdepth.testing import generate_noisy_univariate

from statdepth import FunctionalDepth
from numba import jit

In [4]:
df = generate_noisy_univariate()

bd = FunctionalDepth([df])

In [11]:
@jit(nopython=True)
def test(df):
    return FunctionalDepth([df])

In [12]:
@jit(nopython=True)
def _is_in_simplex(simplex_points: pd.DataFrame, point: pd.Series) -> bool:
    n_points = len(simplex_points)
    n_dim = len(point)
    
    c = np.zeros(n_points)
    A = np.r_[simplex_points.T, np.ones((1, n_points))]
    b = np.r_[point, np.ones(1)]
    
    # This barely ever errors, but when it does the problem is infeasible so let's assume
    # There is no containment
    try:
        lp = linprog(c, A_eq=A, b_eq=b)
    except:
        warnings.warn('Simplex computation failed. Assuming False')
        return False
    
    return lp.success


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'FunctionalDepth': Cannot determine Numba type of <class 'function'>

File "<ipython-input-11-fcc3542ac381>", line 3:
def test(df):
    return FunctionalDepth([df])
    ^

This error may have been caused by the following argument(s):
- argument 0: Cannot determine Numba type of <class 'pandas.core.frame.DataFrame'>


In [10]:
%%timeit
FunctionalDepth([df])

2.64 s ± 194 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
